## Aim of This notebook 

The aim of this notebook is to create two shape file from the tables that we have already in our database.

In [1]:
import pandas as pd
import psycopg2
import credential_temp as  creds 

## Connect to the Database

In [2]:
# connect to our credentioal file
conn_string = "host=" + creds.PGHOST + " port=" + creds.PORT + " dbname=" + creds.PGDATABASE + " user=" + creds.PGUSER \
    + " password=" + creds.PGPASSWORD


In [3]:
conn = psycopg2.connect(conn_string)

## Read the table from database

In this step, we need to read the messstelle table which is already in our database to the pandas dataframe.

In [4]:
quary = """ select * from hygrisc.messstelle """

In [5]:
df = pd.read_sql(quary, conn,)

C:\Users\Sonika.Kaur\AppData\Local\Temp\ipykernel_6644\1697117798.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(quary, conn,)


## Drop the geom column

The table has "geom" column which consist of the geometry information from e32 and n32 columns but we need to delete this column and create this column with another format which can be transfered to geopandas datafarme.   


In [6]:
df.drop('geom', axis = 1, inplace = True)

## Essentioanl packages which need for geopandas dataframe 

In [8]:
import geopandas as gpd
from shapely.geometry import Point

## Create geodata frame

with the helpf of geopandas we can create geo pandas dataframe. in here simply we can create the geometry column again but with another method.

In [9]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.e32, df.n32), crs="EPSG:25832")

In [10]:
gdf

,sl_nr,messstelle_id,name,e32,n32,gw_stockwerk,grundstueck,gemeinde_id,gwhorizont_id,gwhorizont,...,eigentuemer,betreiber,filterlaenge_cm,sumpfrohrlaenge_cm,ausbaudurchmesser_mm,historischer_ruhe_wsp,einbaulaenge_cm,oberkante_filter_cm,unterkante_filter_cm,geometry
0,67530,32505929,UWB-Ddorf 01285,343064,5678019,1.0,,05111000,None,None,...,Stadt Düsseldorf ...,Stadt Düsseldorf ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (343064.000 5678019.000)
1,51044,10446746,60GP012303,292077,5645349,NaN,privat,NL000882,5,Zwischenmittel,...,Prov. Limburg (NL) ...,Prov. Limburg (NL) ...,200.0,NaN,NaN,NaN,16893.0,-3333.0,-3533.0,POINT (292077.000 5645349.000)
2,51070,87005323,58BP024606,287141,5684635,NaN,privat,NL001640,6D,Neurather Sand,...,Prov. Limburg (NL) ...,Prov. Limburg (NL) ...,500.0,300.0,NaN,NaN,32667.0,-29083.0,-29583.0,POINT (287141.000 5684635.000)
3,68442,80000290,Krinsend 0079 neu,309490,5678060,1.0,,05166024,None,None,...,Land NRW ...,None,NaN,NaN,NaN,NaN,1517.0,4832.0,4832.0,POINT (309490.000 5678060.000)
4,68518,86583852,WG102GM93-3,316741,5680237,NaN,privat,05166032,None,None,...,NiederrheinWasser GmbH (ehem. Niederheinwerke)...,NiederrheinWasser GmbH (ehem. Niederheinwerke)...,200.0,NaN,80.0,NaN,5400.0,1413.0,1213.0,POINT (316741.000 5680237.000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71085,69632,76575202,BN Schlachthof EB1,364460,5622115,1.0,,05314000,None,None,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (364460.000 5622115.000)
71086,54639,76562300,JONAS Godorf EB,357178,5635452,NaN,privat,05315000,None,None,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (357178.000 5635452.000)
71087,52815,76678106,WBV Meckenh. EB B,361112,5611396,NaN,privat,05382032,None,None,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (361112.000 5611396.000)
71088,17637,76352201,Zilken Haus Forst,332386,5638906,NaN,privat,05362032,None,None,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (332386.000 5638906.000)


## Create the path of output

In [11]:
data_in_dir = r"../data/shape_file/"

file_fname = r"messstelle_local.gpkg"

file_pfname = data_in_dir + file_fname

## Create the shape file in the above path

In [12]:


gdf.to_file(file_pfname, layer='messstelle_local', driver="GPKG")

## Second shape file.
Now we want to create the second shape file.

* This shape file is a little bit complex than the previous one

## Read the table from Database

We don't want all the rows in messwert table, we want only nitrate and the nitrate stoff number is 1244

In [13]:
quary2 = """ select * from hygrisc.messwert where stoff_nr = '1244' """

In [14]:
df1 = pd.read_sql(quary2, conn,)

C:\Users\Sonika.Kaur\AppData\Local\Temp\ipykernel_6644\3591153979.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1 = pd.read_sql(quary2, conn,)


In [15]:
df1

,messstelle_id,pna_id,datum_pn,stoff_nr,probengut,messergebnis_c,messergebnis_hinweis,bestimmungsgrenze,masseinheit,trennverfahren,verfahren,vor_ort,herkunft,aktual_dat,erstell_dat,messergebnis_cm
0,118262099,11/1988/88122,1988-11-17,1244,Grundwasser,10.1821,None,NaN,mg/l,Nach Laborjournal,DIN 38405-D29 ENTWURF JUNI 1992,,HYGC_BR-MS,1994-12-01,1994-12-01,=
1,118721811,11/1988/88240,1988-10-13,1244,Grundwasser,51.0256,None,NaN,mg/l,Nach Laborjournal,DIN 38405-D29 ENTWURF JUNI 1992,,HYGC_BR-MS,1994-12-01,1994-12-01,=
2,94130723,0/2016/1,2016-05-19,1244,Grundwasser,48.7000,None,NaN,mg/l,Gesamtgehalt,DIN EN ISO 10304-1: Jul 2009 - DO - D20 - 2,,LIMS_LANUV,2021-09-11,2021-09-11,=
3,10403814,7/2019/2,2019-10-28,1244,Grundwasser,6.2000,None,NaN,mg/l,Gesamtgehalt,DIN EN ISO 10304-1 (2009),,HYGRISC_6,2021-09-14,2021-09-14,=
4,10201294,1/1984/2059,1984-10-04,1244,Grundwasser,88.5400,None,NaN,mg/l,Nach Laborjournal,DIN 38405-D29 ENTWURF JUNI 1992,,HYGC_BR-K,1995-01-25,1995-01-25,=
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73894,86592786,3/2018/90284,2018-05-14,1244,Grundwasser,105.3000,None,NaN,mg/l,Gesamtgehalt,DIN EN ISO 10304-1 (1995),,HYGRISC_3,2019-03-08,2019-03-08,=
73895,110151057,0/2018/91169,2018-11-27,1244,Grundwasser,15.0500,None,0.05,mg/l,Gesamtgehalt,DIN ISO 15923-1 - DI - D49 - 1,,LIMS_LANUV,2019-03-12,2019-03-12,=
73896,10202330,7/2021/1,2021-05-25,1244,Grundwasser,58.0000,None,NaN,mg/l,Gesamtgehalt,DIN EN ISO 10304-1 (2009),,HYGRISC_6,2021-08-31,2021-08-31,=
73897,10202341,7/2021/1,2021-05-25,1244,Grundwasser,45.0000,None,NaN,mg/l,Gesamtgehalt,DIN EN ISO 10304-1 (2009),,HYGRISC_6,2021-08-31,2021-08-31,=


## Merge the data

In [16]:
df1_n = df1[[ 'messstelle_id', 'datum_pn', 'stoff_nr', 'messergebnis_c', 'messergebnis_hinweis', 'messergebnis_cm']]

In [17]:
gdf_n = gdf[['messstelle_id', 'name', 'gemeinde_id', 'geometry' ]]

In [18]:
df_merg = pd.merge(df1_n, gdf_n, on = ['messstelle_id'])

In [19]:
df_merg

,messstelle_id,datum_pn,stoff_nr,messergebnis_c,messergebnis_hinweis,messergebnis_cm,name,gemeinde_id,geometry
0,118262099,1988-11-17,1244,10.18210,None,=,WW.LETTE-HUMB.-VK09,05558012,POINT (374250.000 5747750.000)
1,118262099,1988-11-17,1244,10.19999,None,=,WW.LETTE-HUMB.-VK09,05558012,POINT (374250.000 5747750.000)
2,118262099,1983-05-26,1244,3.89576,None,=,WW.LETTE-HUMB.-VK09,05558012,POINT (374250.000 5747750.000)
3,118262099,1983-05-26,1244,3.90001,None,=,WW.LETTE-HUMB.-VK09,05558012,POINT (374250.000 5747750.000)
4,118262099,1983-07-27,1244,4.11711,None,=,WW.LETTE-HUMB.-VK09,05558012,POINT (374250.000 5747750.000)
...,...,...,...,...,...,...,...,...,...
73894,59130532,2017-09-27,1244,1.32810,Konzentration zu gering zur Bestimmung,<,DO-Derne RWI 76,05913000,POINT (396557.000 5714205.000)
73895,91111407,2017-11-14,1244,1.32810,Konzentration zu gering zur Bestimmung,<,Niederaden ML 16,05978024,POINT (402050.000 5715650.000)
73896,80303444,2003-06-23,1244,1.32810,Konzentration zu gering zur Bestimmung,<,AM SCHÖNKES KREUZ F,05166016,POINT (309466.000 5689420.000)
73897,10306377,2004-12-06,1244,1.10675,Konzentration zu gering zur Bestimmung,<,"Stadt Aachen, Nr.63",05334002,POINT (295413.000 5627968.000)


## change the type of "datum_pn" to datetime

In [20]:
df_merg['datum_pn'] = pd.to_datetime(df_merg['datum_pn'], format = '%Y-%m-%d')

In [21]:
df_merg.dtypes

messstelle_id                    int64
datum_pn                datetime64[ns]
stoff_nr                         int64
messergebnis_c                 float64
messergebnis_hinweis            object
messergebnis_cm                 object
name                            object
gemeinde_id                     object
geometry                      geometry
dtype: object

## create the geometry column

In [25]:
gdf_merg = gpd.GeoDataFrame(df_merg, geometry= 'geometry', crs="EPSG:25832")

In [26]:
gdf_merg.dtypes

messstelle_id                    int64
datum_pn                datetime64[ns]
stoff_nr                         int64
messergebnis_c                 float64
messergebnis_hinweis            object
messergebnis_cm                 object
name                            object
gemeinde_id                     object
geometry                      geometry
dtype: object

## Path directory for output

In [27]:
file_fname = r"nitrat_local.gpkg"

file_pfname = data_in_dir + file_fname

## Create shape file

In [28]:
gdf_merg.to_file(file_pfname, layer='nitrat_local', driver="GPKG")

# Finish !
## Good Luck, You are doing well